<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/mmlu_pro/business.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
! pip install -q openai langchain tiktoken faiss-cpu

In [24]:
!pip install -U langchain-upstage langchain-community pypdf

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached pypdf-6.3.0-py3-none-any.whl.metadata (7.1 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.4-py3-none-any.whl.metadata (3.0 kB)


In [25]:
!pip install PDFReader

In [26]:
!pip install jsonlines

In [27]:
UPSTAGE_API_KEY = "up_g7T2cQoLKZH6Oi2n4MHOW706XAdSs"

In [43]:
cd /content/drive/MyDrive/Colab Notebooks/2025_2/nlp

/content/drive/MyDrive/Colab Notebooks/2025_2/nlp


In [29]:
PDF_PATH = "finance.pdf"
OUTPUT_JSONL = "entries_finance.jsonl"
BASE_URL = "https://openstax.org/books/finance-2e"

In [30]:
import os
os.environ["UPSTAGE_API_KEY"] = UPSTAGE_API_KEY

In [31]:
import re
from pathlib import Path

import jsonlines
from pypdf import PdfReader

from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings

In [32]:
CHAPTERS_finance = [
    {"num": 1, "title": "Introduction to Finance", "start": 7,  "end": 36},
    {"num": 2, "title": "Corporate Structure and Governance",     "start": 39, "end": 64},
    {"num": 3, "title": "Economic Foundations: Money and Rates",              "start": 67, "end": 98},
    {"num": 4, "title": "Accrual Accounting Process",    "start": 101, "end": 127},
    {"num": 5, "title": "Financial Statements",    "start": 129, "end": 162},
    {"num": 6, "title": "Measures of Financial Health",    "start": 165, "end": 189},
    {"num": 7, "title": "Time Value of Money I: Single Payment Value",    "start": 191, "end": 225},
    {"num": 8, "title": "Time Value of Money II: Equal Multiple Payments",    "start": 227, "end": 261},
    {"num": 9, "title": "Time Value of Money III: Unequal Multiple Payment Values",    "start": 263, "end": 281},
    {"num": 10, "title": "Bonds and Bond Valuation",    "start": 283, "end": 318},
    {"num": 11, "title": "Stocks and Stock Valuation",    "start": 319, "end": 351},
    {"num": 12, "title": "Historical Performance of US Markets",    "start": 353, "end": 379},
    {"num": 13, "title": "Statistical Analysis in Finance",    "start": 381, "end": 415},
    {"num": 14, "title": "Regression Analysis in Finance",    "start": 417, "end": 447},
    {"num": 15, "title": "How to Think about Investing",    "start": 449, "end": 476},
    {"num": 16, "title": "How Companies Think about Investing",    "start": 479, "end": 504},
    {"num": 17, "title": "How Firms Raise Capital",    "start": 507, "end": 533},
    {"num": 18, "title": "Financial Forecasting",    "start": 535, "end": 563},
    {"num": 19, "title": "The Importance of Trade Credit and Working Capital in Planning",    "start": 565, "end": 597},
    {"num": 20, "title": "Risk Management and the Financial Manager",    "start": 599, "end": 622},
]

In [44]:
CHAPTERS_management = [
    {"num": 1,  "title": "Managing and Performing",
     "start": 7,   "end": 20},
    {"num": 2,  "title": "Managerial Decision-Making",
     "start": 21,  "end": 54},
    {"num": 3,  "title": "The History of Management",
     "start": 55,  "end": 80},
    {"num": 4,  "title": "External and Internal Organizational Environments and Corporate Culture",
     "start": 81,  "end": 118},
    {"num": 5,  "title": "Ethics, Corporate Responsibility, and Sustainability",
     "start": 119, "end": 156},
    {"num": 6,  "title": "International Management",
     "start": 157, "end": 206},
    {"num": 7,  "title": "Entrepreneurship",
     "start": 207, "end": 246},
    {"num": 8,  "title": "Strategic Analysis: Understanding a Firm’s Competitive Environment",
     "start": 247, "end": 278},
    {"num": 9,  "title": "The Strategic Management Process: Achieving and Sustaining Competitive Advantage",
     "start": 279, "end": 304},
    {"num": 10, "title": "Organizational Structure and Change",
     "start": 305, "end": 338},
    {"num": 11, "title": "Human Resource Management",
     "start": 339, "end": 368},
    {"num": 12, "title": "Diversity in Organizations",
     "start": 369, "end": 406},
    {"num": 13, "title": "Leadership",
     "start": 407, "end": 448},
    {"num": 14, "title": "Work Motivation for Performance",
     "start": 449, "end": 496},
    {"num": 15, "title": "Managing Teams",
     "start": 497, "end": 522},
    {"num": 16, "title": "Managerial Communication",
     "start": 523, "end": 548},
    {"num": 17, "title": "Organizational Planning and Controlling",
     "start": 549, "end": 596},
    {"num": 18, "title": "Management of Technology and Innovation",
     "start": 597, "end": 658},  # Index는 659부터
]

In [45]:
CHAPTERS_economics = [
    {"num": 1,  "title": "Welcome to Economics!",                                  "start": 9,   "end": 26},
    {"num": 2,  "title": "Choice in a World of Scarcity",                          "start": 27,  "end": 46},
    {"num": 3,  "title": "Demand and Supply",                                      "start": 47,  "end": 86},
    {"num": 4,  "title": "Labor and Financial Markets",                            "start": 87,  "end": 110},
    {"num": 5,  "title": "Elasticity",                                             "start": 111, "end": 136},
    {"num": 6,  "title": "Consumer Choices",                                       "start": 137, "end": 156},
    {"num": 7,  "title": "Production, Costs, and Industry Structure",              "start": 157, "end": 188},
    {"num": 8,  "title": "Perfect Competition",                                    "start": 189, "end": 216},
    {"num": 9,  "title": "Monopoly",                                               "start": 217, "end": 238},
    {"num": 10, "title": "Monopolistic Competition and Oligopoly",                 "start": 239, "end": 260},
    {"num": 11, "title": "Monopoly and Antitrust Policy",                          "start": 261, "end": 282},
    {"num": 12, "title": "Environmental Protection and Negative Externalities",    "start": 283, "end": 308},
    {"num": 13, "title": "Positive Externalities and Public Goods",                "start": 309, "end": 326},
    {"num": 14, "title": "Labor Markets and Income",                               "start": 327, "end": 360},
    {"num": 15, "title": "Poverty and Economic Inequality",                        "start": 361, "end": 388},
    {"num": 16, "title": "Information, Risk, and Insurance",                       "start": 389, "end": 408},
    {"num": 17, "title": "Financial Markets",                                      "start": 409, "end": 436},
    {"num": 18, "title": "Public Economy",                                         "start": 437, "end": 450},
    {"num": 19, "title": "The Macroeconomic Perspective",                          "start": 451, "end": 480},
    {"num": 20, "title": "Economic Growth",                                        "start": 481, "end": 504},
    {"num": 21, "title": "Unemployment",                                           "start": 505, "end": 532},
    {"num": 22, "title": "Inflation",                                              "start": 533, "end": 560},
    {"num": 23, "title": "The International Trade and Capital Flows",              "start": 561, "end": 584},
    {"num": 24, "title": "The Aggregate Demand/Aggregate Supply Model",            "start": 585, "end": 612},
    {"num": 25, "title": "The Keynesian Perspective",                              "start": 613, "end": 630},
    {"num": 26, "title": "The Neoclassical Perspective",                           "start": 631, "end": 650},
    {"num": 27, "title": "Money and Banking",                                      "start": 651, "end": 670},
    {"num": 28, "title": "Monetary Policy and Bank Regulation",                    "start": 671, "end": 698},
    {"num": 29, "title": "Exchange Rates and International Capital Flows",         "start": 699, "end": 726},
    {"num": 30, "title": "Government Budgets and Fiscal Policy",                   "start": 727, "end": 752},
    {"num": 31, "title": "The Impacts of Government Borrowing",                    "start": 753, "end": 770},
    {"num": 32, "title": "Macroeconomic Policy Around the World",                  "start": 771, "end": 792},
    {"num": 33, "title": "International Trade",                                    "start": 793, "end": 814},
    {"num": 34, "title": "Globalization and Protectionism",                        "start": 815, "end": 844},  # Appendix A가 845부터
]

In [36]:
def extract_pages_text(reader: PdfReader, start_page: int, end_page: int) -> str:
    """0-based 인덱스에서 start_page ~ end_page-1 페이지 텍스트를 이어 붙임."""
    texts = []
    for i in range(start_page, end_page):
        if i < 0 or i >= len(reader.pages):
            continue
        try:
            t = reader.pages[i].extract_text() or ""
        except Exception as e:
            print(f"[WARN] page {i} extract_text 실패: {e}")
            t = ""
        if t.strip():
            texts.append(t)
    return "\n".join(texts)

In [37]:
def normalize_whitespace(text: str) -> str:
    text = text.replace("\t", " ")
    text = "\n".join(line.rstrip() for line in text.splitlines())
    text = re.sub(r"[ ]{2,}", " ", text)
    return text

In [38]:
def split_chapter_into_sections(ch_num: int, chapter_text: str):
    """
    챕터 텍스트를 섹션(1.1, 1.2, ... + Summary/Key Terms 등) 단위로 나눔.
    entries.jsonl 에서 chunk_list 로 들어갈 [{section_title, text}, ...] 반환.
    """
    text = normalize_whitespace(chapter_text)
    lines = text.splitlines()

    header_regex = re.compile(
        rf"^(?:{ch_num}\.\d+\s+.+|Key Terms|Summary|Key Concepts and Summary|"
        r"Self-Check Questions|Review Questions|Critical Thinking Questions|Problems)\s*$"
    )

    headers = []
    for idx, line in enumerate(lines):
        if header_regex.match(line.strip()):
            headers.append((idx, line.strip()))

    if not headers:
        print(f"[WARN] chapter {ch_num} 에서 섹션 헤더를 찾지 못했습니다. 전체를 1개로 처리.")
        return [{
            "section_title": f"Chapter {ch_num} (full)",
            "text": text,
        }]

    sections = []
    for i, (line_idx, header_text) in enumerate(headers):
        start_line = line_idx + 1
        end_line = headers[i + 1][0] if i + 1 < len(headers) else len(lines)
        body_lines = lines[start_line:end_line]
        body = "\n".join(body_lines).strip()
        if not body:
            continue
        sections.append({
            "section_title": header_text,
            "text": body,
        })

    return sections

In [41]:
def split_chapter_into_sections(ch_num: int, chapter_text: str):
    """
    챕터 텍스트를 섹션(1.1, 1.2, ... + Summary/Key Terms 등) 단위로 나눔.
    entries.jsonl 에서 chunk_list 로 들어갈 [{section_title, text}, ...] 반환.
    """
    text = normalize_whitespace(chapter_text)
    lines = text.splitlines()

    header_regex = re.compile(
        rf"^(?:{ch_num}\.\d+\s+.+|Key Terms|Summary|Key Concepts and Summary|"
        r"Self-Check Questions|Review Questions|Critical Thinking Questions|Problems)\s*$"
    )

    headers = []
    for idx, line in enumerate(lines):
        if header_regex.match(line.strip()):
            headers.append((idx, line.strip()))

    if not headers:
        print(f"[WARN] chapter {ch_num} 에서 섹션 헤더를 찾지 못했습니다. 전체를 1개로 처리.")
        return [{
            "section_title": f"Chapter {ch_num} (full)",
            "text": text,
        }]

    sections = []
    for i, (line_idx, header_text) in enumerate(headers):
        start_line = line_idx + 1
        end_line = headers[i + 1][0] if i + 1 < len(headers) else len(lines)
        body_lines = lines[start_line:end_line]
        body = "\n".join(body_lines).strip()
        if not body:
            continue
        sections.append({
            "section_title": header_text,
            "text": body,
        })

    return sections

In [42]:
def create_entries_for_book(pdf_path: str,
                            base_url: str,
                            chapter_specs: list,
                            output_jsonl: str):
    """
    한 책에 대해 entries_*.jsonl 생성.
    JSONL 포맷 (law / psychology와 동일):
      {"source_url": ..., "title": ..., "chunk_list": [{section_title, text}, ...]}
    """
    pdf_path = Path(pdf_path)
    assert pdf_path.exists(), f"PDF가 없습니다: {pdf_path}"

    reader = PdfReader(str(pdf_path))

    with jsonlines.open(output_jsonl, mode="w") as writer:
        for ch in chapter_specs:
            ch_num = ch["num"]
            ch_title = ch["title"]
            start = ch["start"]
            end = ch["end"]

            print(f"[BOOK {pdf_path.name}] CHAPTER {ch_num}: {ch_title} (pages {start}~{end-1})")

            chapter_text = extract_pages_text(reader, start, end)
            sections = split_chapter_into_sections(ch_num, chapter_text)
            print(f"  - 섹션 개수: {len(sections)}")

            entry = {
                "source_url": f"{base_url}#chapter-{ch_num}",
                "title": f"Chapter {ch_num} {ch_title}",
                "chunk_list": sections,
            }
            writer.write(entry)

    print(f"[완료] {output_jsonl} 생성")

In [40]:
upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="embedding-passage")

In [49]:
cd /content/drive/MyDrive/Colab Notebooks/2025_2/nlp

/content/drive/MyDrive/Colab Notebooks/2025_2/nlp


In [52]:
if __name__ == "__main__":
    # 1) Psychology
    create_entries_for_book(
        pdf_path="finance.pdf",
        base_url="https://openstax.org/details/books/principles-finance",
        chapter_specs=CHAPTERS_finance,
        output_jsonl="entries_finance.jsonl",
    )
    # 2) Principles of Management
    create_entries_for_book(
        pdf_path="management.pdf",
        base_url="https://openstax.org/books/principles-management-3e",
        chapter_specs=CHAPTERS_management,
        output_jsonl="entries_management.jsonl",
    )

    # 3) Principles of Economics 3e
    create_entries_for_book(
        pdf_path="economics.pdf",
        base_url="https://openstax.org/books/principles-economics-3e",
        chapter_specs=CHAPTERS_economics,
        output_jsonl="entries_economics.jsonl",
    )

[BOOK finance.pdf] CHAPTER 1: Introduction to Finance (pages 7~35)
  - 섹션 개수: 6
[BOOK finance.pdf] CHAPTER 2: Corporate Structure and Governance (pages 39~63)
  - 섹션 개수: 6
[BOOK finance.pdf] CHAPTER 3: Economic Foundations: Money and Rates (pages 67~97)
  - 섹션 개수: 6
[BOOK finance.pdf] CHAPTER 4: Accrual Accounting Process (pages 101~126)
  - 섹션 개수: 7
[BOOK finance.pdf] CHAPTER 5: Financial Statements (pages 129~161)
  - 섹션 개수: 9
[BOOK finance.pdf] CHAPTER 6: Measures of Financial Health (pages 165~188)
  - 섹션 개수: 7
[BOOK finance.pdf] CHAPTER 7: Time Value of Money I: Single Payment Value (pages 191~224)
  - 섹션 개수: 6
[BOOK finance.pdf] CHAPTER 8: Time Value of Money II: Equal Multiple Payments (pages 227~260)
  - 섹션 개수: 8
[BOOK finance.pdf] CHAPTER 9: Time Value of Money III: Unequal Multiple Payment Values (pages 263~280)
  - 섹션 개수: 3
[BOOK finance.pdf] CHAPTER 10: Bonds and Bond Valuation (pages 283~317)
  - 섹션 개수: 8
[BOOK finance.pdf] CHAPTER 11: Stocks and Stock Valuation (pages 319

In [53]:
#!/usr/bin/env python3
# build_faiss_business_same_meta.py

import jsonlines
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings

# 1) 청킹 설정 (너가 쓰는 값으로 맞춰도 됨)
CHUNK_SIZE = 1600
CHUNK_OVERLAP = 200

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

# 2) 세 책의 entries.jsonl 경로
JSONL_FILES = [
    "entries_psychology.jsonl",
    "entries_management.jsonl",
    "entries_economics.jsonl",
]

all_docs = []

for jsonl_path in JSONL_FILES:
    print(f"Loading {jsonl_path} ...")
    with jsonlines.open(jsonl_path, "r") as reader:
        for entry in reader:
            base_source = entry["source_url"]
            base_title = entry["title"]

            for chunk in entry.get("chunk_list", []):
                section_title = chunk.get("section_title", "")
                section_text = chunk.get("text", "")

                if not section_text.strip():
                    continue

                # 길면 다시 잘게 청킹
                pieces = text_splitter.split_text(section_text)

                for piece in pieces:
                    if not piece.strip():
                        continue

                    # ✅ psychology와 똑같은 메타 포맷
                    meta = {
                        "source_url": base_source,
                        "title": base_title,
                        "section_title": section_title,
                    }

                    all_docs.append(
                        Document(
                            page_content=piece,
                            metadata=meta,
                        )
                    )

print("총 mini-chunk 개수:", len(all_docs))

# 3) Upstage 임베딩 + FAISS 인덱스 생성
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

vectorstore = FAISS.from_documents(all_docs, embeddings)

# 4) index.faiss / index.pkl 저장
INDEX_DIR = "faiss_index_business_all"
vectorstore.save_local(INDEX_DIR)
print(f"[완료] FAISS index saved to: {INDEX_DIR}")

Loading entries_psychology.jsonl ...
Loading entries_management.jsonl ...
Loading entries_economics.jsonl ...
총 mini-chunk 개수: 3523
[완료] FAISS index saved to: faiss_index_business_all
